<a href="https://colab.research.google.com/github/archyyu/GPT-from-MLP-to-RNN-to-Transformer/blob/main/studyLSTM1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import requests
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np

# Set random seed for reproducibility
torch.manual_seed(42)

In [ ]:
# Data I/O
url = "https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt"
#url = "https://raw.githubusercontent.com/archyyu/publicResource/main/google.dev.en"
#url = "https://raw.githubusercontent.com/tinygrad/tinygrad/master/tinygrad/tensor.py"
#url = "https://raw.githubusercontent.com/archyyu/publicResource/main/KDE4.en-es.en"
#url = "https://raw.githubusercontent.com/archyyu/publicResource/main/js"
response = requests.get(url)
data = response.text

chars = list(set(data))
data_size, vocab_size = len(data), len(chars)
print(f'data has {data_size} characters, {vocab_size} unique.')

char_to_ix = {ch: i for i, ch in enumerate(chars)}
ix_to_char = {i: ch for i, ch in enumerate(chars)}

data has 1115394 characters, 65 unique.


In [ ]:
# Hyperparameters
hidden_size = 100
embedding_dim = 50
seq_length = 25
learning_rate = 0.005
batch_size = 20

In [ ]:
import torch
import torch.nn as nn

class LSTMCell(nn.Module):
  def __init__(self, input_size, embedding_size, hidden_size):
    super(LSTMCell, self).__init__()
    self.emb = nn.Embedding(input_size, embedding_size)
    self.W_f = nn.Linear(input_size, hidden_size, bias=False)
    self.h_f = nn.Linear(hidden_size, hidden_size, bias=False)
    self.b_f = nn.Parameter(torch.zeros(hidden_size))

    # Parameters for Input Gate
    self.W_i = nn.Linear(input_size, hidden_size, bias=False)
    self.h_i = nn.Linear(hidden_size, hidden_size, bias=False)
    self.b_i = nn.Parameter(torch.zeros(hidden_size))

    # Parameters for Candidate Cell State
    self.W_C = nn.Linear(input_size, hidden_size, bias=False)
    self.h_C = nn.Linear(hidden_size, hidden_size, bias=False)
    self.b_C = nn.Parameter(torch.zeros(hidden_size))

    # Parameters for Output Gate
    self.W_o = nn.Linear(input_size, hidden_size, bias=False)
    self.h_o = nn.Linear(hidden_size, hidden_size, bias=False)
    self.b_o = nn.Parameter(torch.zeros(hidden_size))

    self.h_o = nn.Linear(hidden_size, input_size, bias=False)
    self.o_b = nn.Parameters(torch.zeros(hidden_size))

  def forward(self, x, h, C_prev):
    x = self.emb(x)
    f_t = torch.sigmoid(self.W_f(x) + self.h_f(h) + self.b_f)
    i_t = torch.sigmoid(self.W_i(x) + self.h_i(h) + self.b_i)
    C_tilde = torch.tanh(self.W_C(x) + self.h_C(h) + self.b_C)
    C_t = f_t * C_prev + i_t * C_tilde
    o_t = torch.sigmoid(self.W_o(x) + self.h_o(h) + self.b_o)
    h_t = o_t * torch.tanh(C_t)

    output = torch.tanh(self.h_o(h_t) + self.o_b)

    return output, h_t, C_t

criterion = nn.CrossEntropyLoss()

model = LSTMCell(vocab_size, embedding_dim, hidden_size)
optimizer = optim.Adagrad(model.parameters(), lr=learning_rate)